In [198]:
import pysal as ps
import numpy as np
import scipy as sp
from gwr.sel_bw import Sel_BW
from gwr.gwr import GWR, MGWR
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels as sm
import statsmodels.genmod.families as fam
from statsmodels.tools import add_constant

In [195]:
import pysal
data = pysal.open('/Users/toshan/Dropbox/dev/GWR4/georgia/GData_utm.csv')
coords = list(zip(data.by_col('X'), data.by_col('Y')))
y = np.array(data.by_col('PctBach')).reshape((-1,1))
rural  = np.array(data.by_col('PctRural')).reshape((-1,1))
pov = np.array(data.by_col('PctPov')).reshape((-1,1)) 
black = np.array(data.by_col('PctBlack')).reshape((-1,1))
fb = np.array(data.by_col('PctFB')).reshape((-1,1))
pop = np.array(data.by_col('TotPop90')).reshape((-1,1))


X = np.hstack([pov, black, rural])


In [241]:

#X = (X - X.mean(axis=0)) / X.std(axis=0)

#y = y.reshape((-1,1))

#y = (y - y.mean(axis=0)) / y.std(axis=0)

sel = Sel_BW(coords, y, X)

bw = sel.search()
print(bw)
r = GWR(coords, y, X, bw, sigma2_v1=False).fit()
print(r.aicc)

93.0
896.349995166


In [50]:
%%time

X = (X - X.mean(axis=0)) / X.std(axis=0)

y = y.reshape((-1,1))

y = (y - y.mean(axis=0)) / y.std(axis=0)

selector = Sel_BW(coords, y, X, multi=True, constant=True)
bw = selector.search()
print(bw)
mgwr = MGWR(coords, y, X, selector, constant=True)
results = mgwr.fit()

[101.0, 101.0, 117.0, 157.0]
CPU times: user 11 s, sys: 1.36 s, total: 12.4 s
Wall time: 3.71 s


In [35]:
selector.bw[0]

[101.0, 101.0, 117.0, 157.0]

In [46]:
results.aicc

297.0704149757866

In [231]:
a = smf.OLS(y, add_constant(X)).fit()
print(np.sqrt(np.sum((y.reshape((-1,1))-a.fittedvalues.reshape((-1,1)))**2)/159.))

4.07443284306


In [180]:
#From GWR4
#-2 log-likelihood:                 860.818394
#Classic AIC:                       892.668583
#AICc:                              896.462831
#ENP (model: trace(S)):                    14.925095
#ENP (variance: trace(S'S)):               10.193958
print(2*14.925095-10.193958)
#ML based sigma estimate:             3.625664 uses ML
#Unbiased sigma estimate:             3.872954 Uses v1v2

19.656232000000003


In [236]:
#From GWmodel
#ML based global sigma estimate:             4.074433
#Unbiased global sigma estimate:             4.126671

#ML based sigma
print(np.sqrt(2106.992 /n))

#v1 based sigma - cant be computed, they don't report v1 (tr(S))
#print(np.sqrt(2106.992 /(n - ))) #v1

#v1v2 based sigma
print(np.sqrt(2106.992 /(n - 18.90946))) #2v1-v2

3.64026400314
3.87817175155


In [184]:
print(r.aic)
print(r.aicc)
print(-2.*r.llf)
print(r.tr_S)
print(r.tr_STS)
print(np.trace(r.S))
print(2*r.tr_S-r.tr_STS)
print(np.sqrt(r.sigma2_ML))
print(np.sqrt(r.sigma2_v1))
print(np.sqrt(r.sigma2_v1v2))


892.824633804
896.349995166
862.096321745
14.3641560298
9.81885125674
14.3641560298
18.9094608029
3.64026393728
3.81674783937
3.8781716925


In [242]:
print(r.bse[0:5])

[[ 2.38331137  0.114245    0.04766552  0.02088569]
 [ 2.54225379  0.12005911  0.05011427  0.02142718]
 [ 2.50507065  0.11746268  0.04973781  0.02129682]
 [ 2.21030762  0.10716526  0.05123175  0.02045047]
 [ 1.74025983  0.10495731  0.04951642  0.01964252]]


In [160]:

n = 159.
k = r.tr_S

In [171]:
#Current GWR AICc - is literally -2 * llf + penalty 
-2.*r.llf + 2.*n*((k+1)/(n-k-2))

896.3499951655665

In [167]:
#From GWR book
2.*n*np.log(np.sqrt(r.sigma2_ML)) + n*np.log(2.*np.pi) + n*((n+k)/(n-k-2))

896.34999516556672

In [164]:
#Just rewriting from the book
n* (np.log(2.*np.pi*r.sigma2_ML) + ((n+k)/(n-k-2.)))

896.34999516556684

In [100]:
#From statsmodels
-2*(-np.log(results.resid_ss)*(n/2.) - (1+np.log(np.pi/(n/2.)))*(n/2.)) 

269.81329106257772

In [101]:
-2*(-n/2 * np.log(2*np.pi*results.sigma2_ML) - (results.resid_ss/(2.*results.sigma2_ML))) + 2.*n*((k+1)/(n-k-2))

297.0704149757866

In [102]:
-2.*results.llf + 2.*n*((k+1)/(n-k-2))

297.0704149757866

In [103]:
-2.*results.llf + 2.*n*((k+1)/(n-k-2))

297.0704149757866

In [104]:
#based on log likelihood
-2*(-n/2 * np.log(2*np.pi*results.sigma2_ML) - (results.resid_ss/(2.*results.sigma2_ML))) + 2.*n*((k+1)/(n-k-2))

297.0704149757866

In [94]:
n*np.log(2*np.pi*results.sigma2_ML) + n +2.*n*((k+1)/(n-k-2))

27006.297617050568

In [95]:
#llf
n*np.log(2*np.pi*results.sigma2_ML) + n +2.*n*((k+1)/(n-k-2))
#book
n*(np.log(results.sigma2_ML*2.*np.pi)) + n*((n+k)/(n-k-2))

27006.297617050568

In [67]:
results.resid_ss / (results.sigma2_ML)

159.0

In [68]:
n +2.*n*((k+1)/(n-k-2))

186.25712391320889

In [69]:
n*((n+k)/(n-k-2))

186.25712391320889

In [92]:
-2.*results.llf + 2*(4+1)

11250.329106257774

In [74]:
results.k

4

In [440]:
n + 

SyntaxError: invalid syntax (<ipython-input-440-c9a9ca7082fc>, line 1)

In [441]:
n*((n+k)/(n-k-2))

193.25367342101035

In [453]:
n*np.log(2*np.pi*results.sigma2_ML) + n +2.*n*((k+1)/(n-k-2))

896.34999516556695

In [449]:
n*(np.log(results.sigma2_ML*2.*np.pi)) + n*((n+k)/(n-k-2))

703.09632174455658

In [7]:
-2.*results.llf + ((2*n*k + 2*n) / (n-k-2))

NameError: name 'results' is not defined

In [446]:
results.aic

892.82463380420779

In [458]:
-2.*results.llf + 2*(results.tr_S+1)

892.82463380420779

In [28]:
results.R2

0.59241530913299911

In [7]:
mgwr.

AttributeError: 'MGWRResults' object has no attribute 'mu'

In [11]:
type(results)

gwr.gwr.MGWRResults

In [12]:
n,k = results.n, 4
CCT = np.zeros((n,k))

In [13]:
for j in range(results.R.shape[2]):

    C = np.dot(np.linalg.inv(np.diag(results.model.X[:,j])), results.R[:,:,j])
    CCT[:,j] = np.diag(np.dot(C,C.T))

In [14]:
CCT.shape

(159, 4)

In [15]:
np.sqrt(CCT*results.sigma2_v1)

array([[ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  

In [157]:
C0 = np.dot(np.linalg.inv(np.diag(results.model.X[:,0])), results.R[:,:,0])
C1 = np.dot(np.linalg.inv(np.diag(results.model.X[:,1])), results.R[:,:,1])
C2 = np.dot(np.linalg.inv(np.diag(results.model.X[:,2])), results.R[:,:,2])
C3 = np.dot(np.linalg.inv(np.diag(results.model.X[:,3])), results.R[:,:,3])


sigma2 = results.sigma2_v1


bse0 = np.sqrt(np.diag(np.dot(C0, C0.T)*sigma2)).reshape((-1,1))
bse1 = np.sqrt(np.diag(np.dot(C1, C1.T)*sigma2)).reshape((-1,1))
bse2 = np.sqrt(np.diag(np.dot(C2, C2.T)*sigma2)).reshape((-1,1))
bse3 = np.sqrt(np.diag(np.dot(C3, C3.T)*sigma2)).reshape((-1,1))




bse = np.hstack([bse0, bse1, bse2, bse3])

In [158]:
bse

array([[ 0.07665169,  0.10936918,  0.06988981,  0.06163678],
       [ 0.07845054,  0.09683867,  0.06937882,  0.06172297],
       [ 0.07705403,  0.11049481,  0.06925705,  0.06161956],
       [ 0.08150328,  0.08015845,  0.07779232,  0.06554729],
       [ 0.07989354,  0.08052692,  0.07126017,  0.06660044],
       [ 0.07712909,  0.07827973,  0.06025873,  0.06641665],
       [ 0.0771068 ,  0.07836167,  0.06002917,  0.06622345],
       [ 0.07624666,  0.07627269,  0.05874826,  0.06623631],
       [ 0.08240539,  0.10654023,  0.06886561,  0.06164584],
       [ 0.0791478 ,  0.08602031,  0.07035505,  0.0619705 ],
       [ 0.0828697 ,  0.07599296,  0.0690824 ,  0.06637866],
       [ 0.08433025,  0.10017878,  0.07121151,  0.06302227],
       [ 0.07899741,  0.11366543,  0.07143428,  0.0621857 ],
       [ 0.08020327,  0.08206745,  0.07251739,  0.06275246],
       [ 0.08008347,  0.11132913,  0.0752955 ,  0.06284956],
       [ 0.08086542,  0.10852503,  0.07686972,  0.06292096],
       [ 0.08191242,  0.

In [12]:
selector.bw[0]

[92.0, 101.0, 136.0, 158.0]

In [6]:
r.w

array([[ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
      

In [16]:
results.sigma2_v1

0.0

In [7]:
results.tr_S

11.36825087269831

In [5]:
results.R2

0.6798781165542741

In [6]:
r.R2

0.46401819400418309

In [8]:
results.resid_response

array([ -1.19971695e-01,  -2.72059429e-01,  -4.79822934e-01,
         3.70219954e-01,  -9.51827921e-02,  -3.31825929e-01,
        -5.61229455e-01,  -2.84968957e-01,  -4.39814593e-01,
        -4.34021065e-01,   5.19487984e-01,  -8.44284863e-02,
        -1.84030082e-01,  -1.86259838e-01,   5.40987832e-01,
         1.55094126e+00,   2.29241167e-01,  -5.56656317e-01,
         4.95041372e-01,   1.49808154e-01,  -2.51380599e-01,
         1.72300023e-01,  -5.32412355e-01,  -5.76875380e-02,
         4.26401297e-01,  -4.18045628e-01,  -3.64861873e-01,
         6.85831175e-01,   1.85877168e+00,   6.34544513e-01,
        -1.99671965e+00,  -4.72795049e-01,   1.25467674e+00,
         1.34713140e-01,  -2.97685577e-01,   2.16768556e-01,
        -7.28527988e-01,   4.99882887e-01,  -6.12780654e-01,
        -7.67652430e-02,  -1.50330021e-01,  -3.41343065e-02,
         1.74388136e-01,  -2.46358410e-01,  -1.61696390e-01,
         6.30447413e-02,   6.49937235e-01,  -6.09604071e-01,
        -1.67828433e-01,

In [15]:
results.resid_ss

50.899379467870425

In [19]:
results.sigma2_v1

0.34477258292171481

In [17]:
results.aicc

297.12013812258783

In [20]:
results.tr_S

11.36825087269831

In [22]:
results.resid_ss

50.899379467870418

In [21]:
results.sigma2_v1

0.34477258292171475

In [20]:
results.aicc

308.91526862334547

In [18]:
results.tr_S

11.368250872698308

In [19]:
results.sigma2_ML

0.3201218834457259

In [9]:
r.aicc

309.99614902284145

In [21]:
print((2.*159*np.log(np.sqrt(r.sigma2_v1))) + (159*np.log(2.*np.pi)) + (159*((159 + r.tr_S)/(159 - 2. - r.tr_S))))


310.704621011


In [17]:
r.tr_S

11.804769716730094

In [5]:
X = (X - X.mean(axis=0)) / X.std(axis=0)

y = y.reshape((-1,1))

y = (y - y.mean(axis=0)) / y.std(axis=0)

sel = Sel_BW(coords, y, X)

bw = sel.search()
print(bw)
results = GWR(coords, y, X, bw, sigma2_v1=True).fit()
n = int(results.n)
sig = np.sqrt(results.sigma2_v1)
print(results.tr_S)
print(sig)
print((2.*n*np.log(sig)) + (n*np.log(2.*np.pi)) + (n*((n + results.tr_S)/(n - 2. - results.tr_S))))


117.0
11.8047697167
0.589697844237
311.31676819


In [138]:
n = 159.
k = results.tr_S

In [139]:


n + ((2.*n*k + 2*n)/(n-2.-k))

186.25712391320889

In [140]:
n*((n+k)/(n-k-2.))

186.25712391320889

In [142]:
n*

310.60505810131076

In [134]:
#llf
n*np.log(2*np.pi*results.sigma2_ML) + n +2.*n*((k+1)/(n-k-2))

280.2054479253228

In [135]:
#book
n*(np.log(results.sigma2_ML*2.*np.pi)) + n*((n+k)/(n-k-2))

280.2054479253228

In [136]:
results.aic

294.76064351384593

In [146]:
-2*results.llf + 2*(k+1) + 2*k*((k+1)/(n-k-1))

296.71413444530072